In [1]:
#Abhängigkeiten
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

## Datensatz einlesen und verarbeiten

In [2]:
#Daten laden
#strip_1 enthält keine near Einträge mit 1.0?
strip_train = pd.read_csv('data/train/strip_1_train.csv', sep=',')
strip_test = pd.read_csv('data/test/strip_1_test_no_labels.csv', sep=',')
strip_offset = 0
#strip_train.head(32)

### Trainingsdaten

In [3]:
#TODO simplify
#Label aus Datensatz entfernen, damit wir einen Label-Satz und einen Feature-Satz erhalten.
X_train = strip_train.drop('near',axis = 1)
Y_train = strip_train['near']

#Alle nicht Sensordaten entfernen
X_train = X_train.drop('frame_number',axis = 1)
X_train = X_train.drop('strip_id',axis = 1)
X_train = X_train.drop('node_id',axis = 1)
X_train = X_train.drop('timestamp',axis = 1)
X_train = X_train.drop('run_number',axis = 1)
X_train = X_train.drop('vicon_x',axis = 1)
X_train = X_train.drop('vicon_y',axis = 1)

#NaN Werte normalisieren
X_train = X_train.fillna(X_train.mean())

#Datenstruktur so ändern, dass wir pro frame eine Zeile mit 150 Sensordaten erhalten
#Anzahl Zeilen / 15 = Frames
frames = len(X_train.index) / 15

trainingData = np.zeros((int(frames),150), dtype=np.float64)
frame_number = 0
node_index = 0
for i, row in X_train.iterrows():
    trainingData[frame_number][node_index * 10 + 0] = row['ax']
    trainingData[frame_number][node_index * 10 + 1] = row['ay']
    trainingData[frame_number][node_index * 10 + 2] = row['az']
    trainingData[frame_number][node_index * 10 + 3] = row['gx']
    trainingData[frame_number][node_index * 10 + 4] = row['gy']
    trainingData[frame_number][node_index * 10 + 5] = row['gz']
    trainingData[frame_number][node_index * 10 + 6] = row['mx']
    trainingData[frame_number][node_index * 10 + 7] = row['my']
    trainingData[frame_number][node_index * 10 + 8] = row['mz']
    trainingData[frame_number][node_index * 10 + 9] = row['r']
    node_index = node_index + 1
    if(node_index >= 15):
        frame_number = frame_number + 1
        node_index = 0
    
#Die Labels Y_train auch auf jede 15te Zeile reduzieren
trainingLabels = np.zeros((int(frames)), dtype=np.int64)
for i, number in Y_train.iteritems():
    if(i % 15 == 0):
        trainingLabels[int(i / 15)] = int(number)
    

In [4]:
#Daten einheitlich skalieren von z. B. 0.0 - 1.0
sc = StandardScaler()
sc.fit(trainingData)
trainingData = sc.transform(trainingData)

### Testdaten

In [5]:
#TODO simplify
#Alle nicht Sensordaten entfernen
X_test = strip_test.drop('frame_number',axis = 1)
X_test = X_test.drop('strip_id',axis = 1)
X_test = X_test.drop('node_id',axis = 1)
X_test = X_test.drop('timestamp',axis = 1)

#NaN Werte normalisieren
X_test = X_test.fillna(X_test.mean())

#Datenstruktur so ändern, dass wir pro frame eine Zeile mit 150 Sensordaten erhalten
#Anzahl Zeilen / 15 = Frames
testFrames = len(X_test.index) / 15

testData = np.zeros((int(testFrames),150), dtype=np.float64)
frame_number = 0
node_index = 0
for i, row in X_test.iterrows():
    testData[frame_number][node_index * 10 + 0] = row['ax']
    testData[frame_number][node_index * 10 + 1] = row['ay']
    testData[frame_number][node_index * 10 + 2] = row['az']
    testData[frame_number][node_index * 10 + 3] = row['gx']
    testData[frame_number][node_index * 10 + 4] = row['gy']
    testData[frame_number][node_index * 10 + 5] = row['gz']
    testData[frame_number][node_index * 10 + 6] = row['mx']
    testData[frame_number][node_index * 10 + 7] = row['my']
    testData[frame_number][node_index * 10 + 8] = row['mz']
    testData[frame_number][node_index * 10 + 9] = row['r']
    node_index = node_index + 1
    if(node_index >= 15):
        frame_number = frame_number + 1
        node_index = 0 

In [6]:
#Daten einheitlich skalieren von z. B. 0.0 - 1.0
sc.fit(testData)
testData = sc.transform(testData)

## Model trainieren

In [7]:
#Random Forest
forest = RandomForestClassifier(n_estimators=200, random_state = 0)
forest.fit(trainingData, trainingLabels)

RandomForestClassifier(n_estimators=200, random_state=0)

## Prediction

In [8]:
forest_prediction = forest.predict(testData)

f = open("data.csv", "w")
f.write("Id,Predicted\n")
count = strip_offset
for a in forest_prediction:
    f.write(str(count))
    f.write(",")
    f.write(str(int(a)))
    f.write("\n")
    count = count + 1 
f.close()